<a href="https://colab.research.google.com/github/ericburdett/cs474_labs_f2019/blob/master/DL_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/ericburdett/cs474_labs_f2019/blob/master/DL_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3: Intro to CNNs and DNNs

## Objectives

* Build and train a deep conv net
* Explore and implement various initialization techniques
* Implement a parameterized module in Pytorch
* Use a principled loss function

## Video Tutorial
[https://youtu.be/3TAuTcx-VCc](https://youtu.be/3TAuTcx-VCc)

## Deliverable
For this lab, you will submit an ipython notebook via learningsuite.
This is where you build your first deep neural network!

For this lab, we'll be combining several different concepts that we've covered during class,
including new layer types, initialization strategies, and an understanding of convolutions.

## Grading Standards:
* 30% Part 0: Successfully followed lab video and typed in code
* 20% Part 1: Re-implement Conv2D and CrossEntropy loss function
* 20% Part 2: Implement different initialization strategies
* 10% Part 3: Print parameters, plot train/test accuracy
* 10% Part 4: Convolution parameters quiz
* 10% Tidy and legible figures, including labeled axes where appropriate
___

### Part 0
Watch and follow video tutorial:

[https://youtu.be/3TAuTcx-VCc](https://youtu.be/3TAuTcx-VCc)

**TODO:**


**DONE:**
* Watch tutorial

In [0]:
!pip3 install torch
!pip3 install torchvision
!pip3 install tqdm

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb

assert torch.cuda.is_available(), "You need to request a GPU from Runtime > Change Runtime"

In [0]:
# Use the dataset class you created in lab2

class Conv2d(nn.Module):
  def __init__(self, n_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True):
    self.__dict__.update(locals())
    super(Conv2d, self).__init__()
    
    self.weight = Parameter(torch.Tensor(out_channels,
                                         n_channels,
                                         *kernel_size))
    self.bias = Parameter(torch.Tensor(out_channels))
    
    self.weight.data.uniform_(-1, 1)
    self.bias.data.uniform_(0, 0)
    
  def forward(self, x):
    return F.conv2d(x, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups)
                                

class ConvNetwork(nn.Module):
  def __init__(self, dataset):
    super(ConvNetwork, self).__init__()
    x, y = dataset[0]
    c, h, w = x.size()
    output = 10
    
    self.net = nn.Sequential(
      Conv2d(c, 10, (3, 3), padding=(1, 1)),
      Conv2d(10, output, (28, 28), padding=(0, 0))
    )
    
  def forward(self, x):
    return self.net(x).squeeze(2).squeeze(2)
    

class FashionMNISTProcessedDataset(Dataset):
  def __init__(self, root, train=True):
    self.data = datasets.FashionMNIST(root,
                                     train=train,
                                     transform=transforms.ToTensor(),
                                     download=True)
  def __getitem__(self, i):
    x, y = self.data[i]
    # pdb.set_trace() # for debugging... stops at this point in execution and you can check data types, vars, etc.
    return x, y
  
  def __len__(self):
    # Consider modifying this value for future parts
    return len(self.data)
  
  
train_dataset = FashionMNISTProcessedDataset('/tmp/fashionmnist', train=True)
val_dataset = FashionMNISTProcessedDataset('/tmp/fashionmnist', train=False)


model = ConvNetwork(train_dataset)
model = model.cuda()

objective = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

train_loader = DataLoader(train_dataset,
                          batch_size=42,
                          pin_memory=True)

val_loader = DataLoader(val_dataset,
                        batch_size=42,
                        pin_memory=True)

train_losses = []
val_losses = []

for epoch in range(1):
  loop = tqdm(total=len(train_loader), position=0, leave=False)
  
  for batch, (x, y_truth) in enumerate(train_loader):
    x, y_truth = x.cuda(async=True), y_truth.cuda(async=True)
    
    optimizer.zero_grad()
    y_hat = model(x)
    
    loss = objective(y_hat, y_truth)
    
    loss.backward()
    
    train_losses.append(loss.item())
    accuracy = 0
    loop.set_description('epoch:{}, loss:{:.4f}'.format(epoch, loss))
    loop.update(1)
    
    optimizer.step()
    
    if batch % 100 == 0:
      val = np.mean([objective(model(x.cuda()), y.cuda()).item()
                    for x, y in val_loader])
      val_losses.append((len(train_losses), val))

  loop.close()
  
a, b = zip(*val_losses)
plt.plot(train_losses, label='train')
plt.plot(a, b, label='val')
plt.legend()
plt.show()


___

### Part 1
Re-implement a Conv2D module with parameters and a CrossEntropy loss function.

**TODO:**


**DONE:**
* CrossEntropyLoss 
* Conv2D
___

### Part 2
Implement a few initialization strategies which can include Xe initialization
(sometimes called Xavier), Orthogonal initialization, and uniform random.
You can specify which strategy you want to use with a parameter. 



Helpful links include:
*  [Orthogonal Initialization](https://hjweide.github.io/orthogonal-initialization-in-convolutional-layers) (or the original paper: http://arxiv.org/abs/1312.6120)
*  http://andyljones.tumblr.com/post/110998971763/an-explanation-of-xavier-initialization

**TODO:**


**DONE:**
* Parameterize custom Conv2D for different initilization strategies
* Xe
* Orthogonal
* Uniform


In [0]:
a = torch.from_numpy(np.random.randn(3,4,1).astype(np.float32))
b = torch.exp(a)
b[:,].size()

torch.Size([3, 4, 1])

In [0]:
class CrossEntropyLoss(nn.Module):
  def __init__(self, weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean'):
    super(CrossEntropyLoss, self).__init__()
  
  def forward(self, y_hat, y_truth):
    wrong_class_penalty = torch.log(torch.sum(torch.exp(y_hat), dim=1))
    n, classes = y_hat.size()
    b = torch.zeros((n, classes))
    b[np.arange(n), y_truth] = 1
    true_class_preds = torch.sum(y_hat * b.cuda(), dim=1)
    return torch.mean(-true_class_preds + wrong_class_penalty)

class Conv2d(nn.Module):
  def __init__(self, n_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, init='xe'):
    self.__dict__.update(locals())
    super(Conv2d, self).__init__()
    
    self.weight = Parameter(torch.Tensor(out_channels,
                                         n_channels,
                                         *kernel_size))
    
    if init == 'uniform':
      self.weight.data.uniform_(-1, 1)
    elif init == 'xe':
      nn.init.xavier_uniform_(self.weight.data)
    elif init == 'orthogonal':
      U, S, V = torch.svd(self.weight.data, some=True)
      torch.allclose(torch.dot(V, V.T), V.shape[0])
      self.weight.data = V.reshape((n_channels, out_channels, kernel_size[0], kernel_size[1]))
    
    
    self.bias = Parameter(torch.Tensor(out_channels))
    self.bias.data.uniform_(0, 0)
    
  def forward(self, x):
    return F.conv2d(x, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups)


In [0]:
class ConvNetwork(nn.Module):
  def __init__(self, dataset):
    super(ConvNetwork, self).__init__()
    x, y = dataset[0]
    c, h, w = x.size()
    output = 10
    
    #pdb.set_trace()
    
    self.net = nn.Sequential(
      Conv2d(c, 30, (3, 3), padding=(1, 1), init='xe'),
      nn.ReLU(),
      Conv2d(30, 45, (5, 5), padding=(2, 2), init='xe'),
      nn.ReLU(),
      Conv2d(45, 150, (9, 9), padding=(4, 4), init='xe'),
      nn.ReLU(),
      Conv2d(150, 92, (11, 11), padding=(5, 5), init='xe'),
      nn.ReLU(),
      Conv2d(92, 350, (15, 15), padding=(7, 7), init='xe'),
      nn.ReLU(),
      Conv2d(350, output, (28, 28), padding=(0, 0), init='xe'),
      nn.ReLU()
    )
    
  def forward(self, x):
    return self.net(x).squeeze(2).squeeze(2)


In [0]:
train_dataset = FashionMNISTProcessedDataset('/tmp/fashionmnist', train=True)
val_dataset = FashionMNISTProcessedDataset('/tmp/fashionmnist', train=False)


model = ConvNetwork(train_dataset)
model = model.cuda()

objective = CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

train_loader = DataLoader(train_dataset,
                          batch_size=42,
                          pin_memory=True)

val_loader = DataLoader(val_dataset,
                        batch_size=42,
                        pin_memory=True)

train_losses = []
val_losses = []

for epoch in range(1):
  loop = tqdm(total=len(train_loader), position=0, leave=False)
  
  for batch, (x, y_truth) in enumerate(train_loader):
    x, y_truth = x.cuda(async=True), y_truth.cuda(async=True)
    
    optimizer.zero_grad()
    y_hat = model(x)
    
    loss = objective(y_hat, y_truth)
    
    loss.backward()
    
    train_losses.append(loss.item())
    accuracy = 0
    loop.set_description('epoch:{}, loss:{:.4f}'.format(epoch, loss))
    loop.update(1)
    
    optimizer.step()
    
    if batch % 100 == 0:
      val = np.mean([objective(model(x.cuda()), y.cuda()).item()
                    for x, y in val_loader])
      val_losses.append((len(train_losses), val))

  loop.close()
  
a, b = zip(*val_losses)
plt.plot(train_losses, label='train')
plt.plot(a, b, label='val')
plt.legend()
plt.show()


epoch:0, loss:2.1025:   7%|▋         | 101/1429 [00:56<06:24,  3.45it/s]


___

### Part 3
Print the number of parameters in your network and plot accuracy of your training and validation 
set over time. You should experiment with some deep networks and see if you can get a network 
with close to 1,000,000 parameters.

**TODO:**
* Experiment with Deep Networks
* Plot accuracy of training and validation set over time
* Print out number of parameters in the model 

**DONE:**


In [0]:

# Go back up and try a few different networks and initialization strategies
# Plot loss if you want
# Plot accuracy



In [136]:
for p in model.parameters():
  print(p.size())

torch.Size([30, 1, 3, 3])
torch.Size([30])
torch.Size([45, 30, 3, 3])
torch.Size([45])
torch.Size([10, 45, 3, 3])
torch.Size([10])
torch.Size([10, 10, 28, 28])
torch.Size([10])


In [137]:
# Compute and print the number of parameters in the model

total_params = 0
for p in model.parameters():
  
  layer_params = 1
  
  for s in p.size():
    layer_params *= s
  
  total_params += layer_params
  
total_params

94965

___

### Part 4
Learn about how convolution layers affect the shape of outputs, and answer the following quiz questions. Include these in a new markdown cell in your jupyter notebook.


*Using a Kernel size of 3×3 what should the settings of your 2d convolution be that results in the following mappings (first answer given to you)*

* (c=3, h=10, w=10) ⇒ (c=10, h=8, w=8) : (out_channels=10, kernel_size=(3, 3), padding=(0, 0))
* (c=3, h=10, w=10) ⇒ (c=22, h=10, w=10) : **Your answer in bold here**
* (c=3, h=10, w=10) ⇒ (c=65, h=12, w=12) : **Your answer in bold here**
* (c=3, h=10, w=10) ⇒ (c=7, h=20, w=20) : **Your answer in bold here**

*Using a Kernel size of 5×5:*)

* (c=3, h=10, w=10) ⇒ (c=10, h=8, w=8) : (out_channels=10, kernel_size=(5, 5), padding=(1, 1))
* (c=3, h=10, w=10) ⇒ (c=100, h=10, w=10) : **Your answer in bold here**
* (c=3, h=10, w=10) ⇒ (c=23, h=12, w=12) : **Your answer in bold here**
* (c=3, h=10, w=10) ⇒ (c=5, h=24, w=24) : **Your answer in bold here**

*Using Kernel size of 5×3:*

* (c=3, h=10, w=10) ⇒ (c=10, h=8, w=8) : **Your answer in bold here**
* (c=3, h=10, w=10) ⇒ (c=100, h=10, w=10) : **Your answer in bold here**
* (c=3, h=10, w=10) ⇒ (c=23, h=12, w=12) : **Your answer in bold here**
* (c=3, h=10, w=10) ⇒ (c=5, h=24, w=24) : **Your answer in bold here**

*Determine the kernel that requires the smallest padding size to make the following mappings possible:*

* (c=3, h=10, w=10) ⇒ (c=10, h=9, w=7) : **Your answer in bold here**
* (c=3, h=10, w=10) ⇒ (c=22, h=10, w=10) : **Your answer in bold here**

**TODO:**

* Answer all the questions above 

**DONE:**


In [0]:
# Write some test code for checking the answers for these problems (example shown in the video)
